In [1]:
%pip install langchain langchain_openai wikipedia langchain-community --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

- The name of the tool
- A description of what the tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the result of a tool should be returned directly to the user

It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [4]:
# 1. Standard Tools
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.name)
print(tool.description)
print(tool.args)

# We can see if the tool should return directly to the user
print("Will this automatically return the output to the user? This value is a boolean:", tool.return_direct)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'title': 'Query', 'type': 'string'}}
Will this automatically return the output to the user? This value is a boolean: False


## How to Call The Tool Without An Agent:

In [5]:
tool.invoke("What is Digital Marketing?")

'Page: Digital marketing\nSummary: Digital marketing is the component of marketing that uses the Internet and online-based digital technologies such as desktop computers, mobile phones, and other digital media and platforms to promote products and services. Its development during the 1990s and 2000s changed the way brands and businesses use technology for marketing. As digital platforms became increasingly incorporated into marketing plans and everyday life, and as people increasingly used digital devices instead of visiting physical shops, digital marketing campaigns have become prevalent, employing combinations of search engine optimization (SEO), search engine marketing (SEM), content marketing, influencer marketing, content automation, campaign marketing, data-driven marketing, e-commerce marketing, social media marketing, social media optimization, e-mail direct marketing, display advertising, e-books, and optical disks and games have become commonplace. Digital marketing extends 

# 2. Creating Custom Tools
When constructing your own agent, you will need to provide it with a list of Tools that it can use. Besides the actual function that is called, the Tool consists of several components:

- `name (str)`, is required and must be unique within a set of tools provided to an agent
- `description (str)`, is optional but recommended, as it is used by an agent to determine tool use
- `args_schema (Pydantic BaseModel)`, is optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.

In [6]:
# Import things that are needed generically
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

## `@tool decorator`

This `@tool` decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function’s docstring as the tool’s description - so a docstring MUST be provided.

In [7]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


You can also customize the tool name and JSON args by passing them into the tool decorator.

In [9]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

## StructuredTool dataclass

In [10]:
def search_function(query: str):
    return "LangChain"

search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    # coroutine= ... <- you can specify an async method if desired as well
)

--------

## [Agents](https://python.langchain.com/docs/modules/agents/)

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

There are lots of different [types of agents](https://python.langchain.com/docs/modules/agents/agent_types/) that LangChain offer.

In [11]:
from langchain.agents import tool

# 1. Create the tool:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# 2. Assign the tools to a Python list:
tools = [get_word_length]

In [14]:
# 3. Create the ChatPromptTemplate:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Bind tools to LLM

How does the agent know what tools it can use?

In this case we’re relying on OpenAI tool calling LLMs, which take tools as a separate argument and have been specifically trained to know when to invoke those tools.

In [15]:
from langchain_openai import ChatOpenAI

# 4. Create the LLM and bind the tools directly to the LLM:
llm = ChatOpenAI(model='gpt-4-turbo')
llm_with_tools = llm.bind_tools(tools=tools)

## Create the Agent

Putting those pieces together, we can now create the agent. We will import two last utility functions: a component for formatting intermediate steps (agent action, tool output pairs) to input messages that can be sent to the model, and a component for converting the output message into an agent action/agent finish.

In [16]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# 5. Creating the LCEL agent chain:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
list(agent_executor.stream( {"input": "How many letters in the word data"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'data'}`


4The word "data" has 4 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'data'}, log="\nInvoking: `get_word_length` with `{'word': 'data'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_BFTwdEOwbhmh2cXNRBdBrhop', 'function': {'arguments': '{"word":"data"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-ec1b1c17-a4b2-494f-be4a-34a7c11dec66', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'data'}, 'id': 'call_BFTwdEOwbhmh2cXNRBdBrhop'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"data"}', 'id': 'call_BFTwdEOwbhmh2cXNRBdBrhop', 'index': 0}])], tool_call_id='call_BFTwdEOwbhmh2cXNRBdBrhop')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_BFTwdEOwbhmh2cXNRBdBrhop', 'function': {'arguments': '{"word":"data"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'fin

In [19]:
agent_executor.invoke({"input": "How many letters in the word data"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'data'}`


4The word "data" has 4 letters.

> Finished chain.


{'input': 'How many letters in the word data',
 'output': 'The word "data" has 4 letters.'}

## Adding in Memory

This is great - we have an agent! However, this agent is stateless - it doesn’t remember anything about previous interactions. This means you can’t ask follow up questions easily. Let’s fix that by adding in memory.

In order to do this, we need to do two things:

Add a place for memory variables to go in the prompt
Keep track of the chat history
First, let’s add a place for memory in the prompt. We do this by adding a placeholder for messages with the key "chat_history". Notice that we put this ABOVE the new user input (to follow the conversation flow).

In [20]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [21]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [22]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [23]:
input1 = "how many letters in the word data?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'data'}`


4The word "data" has 4 letters.

> Finished chain.


> Entering new AgentExecutor chain...
Yes, "data" is a real word. It refers to facts and statistics collected together for reference or analysis.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word data?'),
  AIMessage(content='The word "data" has 4 letters.')],
 'output': 'Yes, "data" is a real word. It refers to facts and statistics collected together for reference or analysis.'}

## Customizing the memory by wrapping it with a `specific session_id`:

In [24]:
# Customising the memory by 
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name='history'),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "history": lambda x: x.get("history", []),
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [25]:
with_message_history.invoke(
    {"input": "My name is James", "history": []},
    config={"configurable": {"session_id": "some_session_id"}},
)



> Entering new AgentExecutor chain...
Hello James! How can I assist you today?

> Finished chain.


{'input': 'My name is James',
 'history': [],
 'output': 'Hello James! How can I assist you today?'}

In [27]:
with_message_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "some_session_id"}},
)



> Entering new AgentExecutor chain...
Your name is James. How can I help you further?

> Finished chain.


{'input': 'What is my name?',
 'history': [HumanMessage(content='My name is James'),
  AIMessage(content='Hello James! How can I assist you today?')],
 'output': 'Your name is James. How can I help you further?'}

In [28]:
with_message_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "some_different_session_id"}},
)



> Entering new AgentExecutor chain...
You haven't mentioned your name yet. Please tell me your name, and I'll remember it!

> Finished chain.


{'input': 'What is my name?',
 'history': [],
 'output': "You haven't mentioned your name yet. Please tell me your name, and I'll remember it!"}